## Loading the Data

DO NOT FORGET TO SEPARATE A TEST SET!!!

In [1]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files='/home/j/Documents/Projects/MLotsawa/notebooks/data collection and cleaning/train-pairs.csv')

In [2]:
dataset = dataset['train'].train_test_split(.15)

In [4]:
dataset['train'][0]

{'Unnamed: 0': 19887,
 'tibetan': 'དེ་ནས་ཉེར་ཐོབ་གཞི་ཡི་འོད་གསལ་ནི།།',
 ' phonetic': 'dené nyertob zhi yi ösal ni',
 ' english': 'Then when the ground luminosity of near-attainment dawns'}

## Load Unfinetuned Tokenizer, Model, and Data Collator

In [5]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM

checkpoint = "billingsmoore/phonetic-tibetan-to-english-translation"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map="cuda:0")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

2024-09-26 15:22:57.560393: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-26 15:22:57.560447: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-26 15:22:57.565768: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-26 15:22:58.252172: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


tokenizer_config.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

## Add Tibetan to Tokenizer

The T5 tokenizer does not notably support the Tibetan script. So, we need to add it manually. Once the characters have been added to the tokenizer, the model needs to have its token embeddings resized to accomodate the added tokens. This is all pretty straightforward, as seen in the code below.

In [6]:
# Tibetan characters to add
tibetan_chars = [
    # Consonants
    "ཀ", "ཁ", "ག", "ང", "ཅ", "ཆ", "ཇ", "ཉ", "ཏ", "ཐ", "ད", "ན", "པ", "པ", "ཕ", "བ", "མ",
    "ཙ", "ཚ", "ཛ", "ཝ", "ཞ", "ཟ", "འ", "ཡ", "ར", "ལ", "ཤ", "ཥ", "ས", "ཧ", "ཨ",

    # Subjoined Consonants
    "ྐ", "ྑ", "ྒ", "ྒྷ", "ྔ", "ྕ", "ྖ", "ྗ", "྘", "ྙ", "ྚ", "ྛ", "ྜ", "ྜྷ", "ྞ", "ྟ",
    "ྠ", "ྡ", "ྡྷ", "ྣ", "ྤ", "ྥ", "ྦ", "ྦྷ", "ྨ", "ྩ", "ྪ", "ྫ", "ྫྷ", "ྭ", "ྮ", "ྯ",
    "ྰ", "ྱ", "ྲ", "ླ", "ྴ", "ྵ", "ྶ", "ྷ", "ྸ", "ྐྵ", "ྺ", "ྻ", "ྼ", "྽", "྾", "྿",

    # Vowels
    "ི", "ཱི", "ུ", "ཱུ", "ྲྀ", "ཷ", "ླྀ", "ཹ", "ེ", "ཻ", "ོ", "ཽ", "ཾ", "ཿ",

    # Other Marks and Symbols
    "འ", "ཡ", "ར", "ལ", "ཤ", "ཥ", "ས", "ཧ", "ཨ",

    # Additional Tibetan Characters
    "ཀྵ", "ཁྵ", "གྵ", "ངྵ", "ཅྵ", "ཆྵ", "ཇྵ", "ཉྵ", "ཏྵ", "ཐྵ", "དྵ", "ནྵ", "པྵ", 
    "པྵ", "ཕྵ", "བྵ", "མྵ", "ཙྵ", "ཚྵ", "ཛྵ", "ཝྵ", "ཞྵ", "ཟྵ", "འྵ", "ཡྵ", "རྵ", 
    "ལྵ", "ཤྵ", "ཥྵ", "སྵ", "ཧྵ", "ཨྵ", "པྪ", "པྫ", "པྫྷ", "པྭ", "པྮ", "པྯ", "པྰ", 
    "པྱ", "པྲ", "པླ", "པྴ", "པྵ", "པྶ", "པྷ", "པྸ", "པྐྵ", "པྺ", "པྻ", "པྼ", "པ྽", 
    "པ྾", "པ྿"
]


#'ཀཁགངཅཆཇཉཏཐདནཔཕབམཙཚཛཝཞཟའཡརལཤཥསཧཨ'

# Add the Tibetan characters to the tokenizer's vocabulary
new_tokens = [char for char in tibetan_chars if char not in tokenizer.get_vocab()]

# Add new tokens to the tokenizer
tokenizer.add_tokens(new_tokens)

# Resize model embeddings to accommodate the new vocabulary size
model.resize_token_embeddings(len(tokenizer))

Embedding(32245, 1024)

## Preprocess Data

The dataset can now be tokenized for training.

In [7]:
source_lang = 'tibetan'
target_lang = ' english'

def preprocess_function(examples):

    inputs = [example for example in examples[source_lang]]
    targets = [example for example in examples[target_lang]]
    
    model_inputs = tokenizer(inputs, text_target=targets, max_length=256, truncation=True, padding="max_length")

    return model_inputs

In [8]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/69042 [00:00<?, ? examples/s]

Map:   0%|          | 0/12184 [00:00<?, ? examples/s]

## Define Metric

In [ ]:
import numpy as np

import evaluate

metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

## Train the Model

Finally, we can train the model. Note that the optimizer used is Adafactor. This is the optimizer that is preferred for translation tasks and for the T5 model in general. The transformers api includes a built in version of Adafactor, but I define it separately here so that we can optimize it with the 'accelerate' library.

In [9]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, Adafactor

optimizer = Adafactor(
    model.parameters(), 
    scale_parameter=True, 
    relative_step=False, 
    warmup_init=False, 
    lr=2e-5
)

In [10]:
from accelerate import Accelerator

accelerator = Accelerator()

optimizer = Adafactor(
    model.parameters(), 
    scale_parameter=True, 
    relative_step=False, 
    warmup_init=False, 
    lr=3e-4
)

model, optimizer = accelerator.prepare(model, optimizer)

In [11]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"pre-trained",
    auto_find_batch_size=True,
    predict_with_generate=True,
    fp16=False, #check this
    push_to_hub=False,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    num_train_epochs=3
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['train'],
    tokenizer=tokenizer,
    optimizers=(optimizer, None),
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: billingsmoore. Use `wandb login --relogin` to force relogin


  0%|          | 0/25893 [00:00<?, ?it/s]

  0%|          | 0/51783 [00:00<?, ?it/s]